In [249]:
import numpy as np
import pandas as pd
import json, requests

#get json data
f = open('orders.json')
q1_orders = json.load(f)
f.close()

#get api exchange rates
response = requests.get('https://api.exchangeratesapi.io/history?start_at=2020-01-01&end_at=2020-03-31&base=USD&symbols=CAD')
response.raise_for_status()
fx = json.loads(response.text)


#join on date
#create two dataframes
df_q1 = pd.json_normalize(q1_orders)
#df_q1 = pd.DataFrame(q1_orders)
df_fx = pd.DataFrame(fx)






In [250]:
df_fx.head()

,rates,start_at,base,end_at
2020-01-02,{'CAD': 1.299830251},2020-01-01,USD,2020-03-31
2020-01-03,{'CAD': 1.2981968243},2020-01-01,USD,2020-03-31
2020-01-06,{'CAD': 1.2975701269},2020-01-01,USD,2020-03-31
2020-01-07,{'CAD': 1.2996777658},2020-01-01,USD,2020-03-31
2020-01-08,{'CAD': 1.3018443545},2020-01-01,USD,2020-03-31


In [251]:
df_fx['cad_rates'] = df_fx['rates'].apply(lambda x: x['CAD'])
df_fx.head()

,rates,start_at,base,end_at,cad_rates
2020-01-02,{'CAD': 1.299830251},2020-01-01,USD,2020-03-31,1.299830
2020-01-03,{'CAD': 1.2981968243},2020-01-01,USD,2020-03-31,1.298197
2020-01-06,{'CAD': 1.2975701269},2020-01-01,USD,2020-03-31,1.297570
2020-01-07,{'CAD': 1.2996777658},2020-01-01,USD,2020-03-31,1.299678
2020-01-08,{'CAD': 1.3018443545},2020-01-01,USD,2020-03-31,1.301844


In [252]:
df_fx_cad = df_fx['cad_rates']
df_fx_cad.loc['2020-03-01':'2020-03-31']

2020-03-02    1.335821
2020-03-03    1.336422
2020-03-04    1.335910
2020-03-05    1.342719
2020-03-06    1.342008
2020-03-09    1.362867
2020-03-10    1.367779
2020-03-11    1.374294
2020-03-12    1.381139
2020-03-13    1.385897
2020-03-16    1.392758
2020-03-17    1.412493
2020-03-18    1.440369
2020-03-19    1.446440
2020-03-20    1.424956
2020-03-23    1.438375
2020-03-24    1.449414
2020-03-25    1.435208
2020-03-26    1.408433
2020-03-27    1.413956
2020-03-30    1.416168
2020-03-31    1.425429
Name: cad_rates, dtype: float64

In [253]:
df_q1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20 non-null     int64  
 1   total_price     20 non-null     float64
 2   created_at      20 non-null     object 
 3   line_items      20 non-null     object 
 4   customer.id     20 non-null     int64  
 5   customer.name   20 non-null     object 
 6   customer.email  20 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.2+ KB


In [254]:
df_q1.describe()

,id,total_price,customer.id
count,2.000000e+01,20.000000,20.000000
mean,5.246519e+15,108.213500,57084.250000
std,2.628864e+15,102.581416,26168.695474
min,1.104846e+15,5.990000,23955.000000
25%,2.337426e+15,36.990000,41372.250000
50%,5.549575e+15,67.685000,59933.000000
75%,7.356313e+15,100.975000,68629.750000
max,9.786846e+15,315.970000,94720.000000


In [255]:
df_q1['order_date'] = df_q1['created_at'].apply(lambda x: x[:10])

In [256]:
df_join_q1_fx = pd.merge(left=df_q1,right=df_fx_cad,left_on='order_date',right_index=True,how='left')

In [257]:
df_q1.head()

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01
2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10
4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15


In [258]:
df_fx_cad.head()

2020-01-02    1.299830
2020-01-03    1.298197
2020-01-06    1.297570
2020-01-07    1.299678
2020-01-08    1.301844
Name: cad_rates, dtype: float64

In [259]:
df_join_q1_fx.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20 non-null     int64  
 1   total_price     20 non-null     float64
 2   created_at      20 non-null     object 
 3   line_items      20 non-null     object 
 4   customer.id     20 non-null     int64  
 5   customer.name   20 non-null     object 
 6   customer.email  20 non-null     object 
 7   order_date      20 non-null     object 
 8   cad_rates       10 non-null     float64
dtypes: float64(2), int64(2), object(5)
memory usage: 1.5+ KB


In [260]:
df_join_q1_fx

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date,cad_rates
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07,NaN
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01,NaN
2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01,NaN
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10,1.367779
4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15,NaN
5,6431399815998774,315.97,2020-03-12T07:16:07Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-12,1.381139
6,5791957346866373,36.99,2020-03-24T09:52:40Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-24,1.449414
7,2079640804901496,55.98,2020-02-19T19:33:57Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-02-19,1.322315
8,2249838395290890,309.98,2020-03-13T12:15:51Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-13,1.385897
9,7638742701884072,35.39,2020-03-08T05:28:01Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-08,NaN


In [261]:
df_join_q1_fx['cad_rates'].fillna(df_join_q1_fx['cad_rates'].mean(),inplace =True)
df_join_q1_fx

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date,cad_rates
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07,1.381052
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01,1.381052
2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01,1.381052
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10,1.367779
4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15,1.381052
5,6431399815998774,315.97,2020-03-12T07:16:07Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-12,1.381139
6,5791957346866373,36.99,2020-03-24T09:52:40Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-24,1.449414
7,2079640804901496,55.98,2020-02-19T19:33:57Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-02-19,1.322315
8,2249838395290890,309.98,2020-03-13T12:15:51Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-13,1.385897
9,7638742701884072,35.39,2020-03-08T05:28:01Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-08,1.381052


In [262]:
df_join_q1_fx['total_price_cad'] = round((df_join_q1_fx['total_price'] * df_join_q1_fx['cad_rates']),2)
df_join_q1_fx.reset_index(inplace=True)

In [264]:
df_join_q1_fx

,index,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,order_date,cad_rates,total_price_cad
0,0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07,1.381052,109.64
1,1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01,1.381052,120.12
2,2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01,1.381052,51.09
3,3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10,1.367779,76.57
4,4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15,1.381052,428.10
5,5,6431399815998774,315.97,2020-03-12T07:16:07Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-12,1.381139,436.40
6,6,5791957346866373,36.99,2020-03-24T09:52:40Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-24,1.449414,53.61
7,7,2079640804901496,55.98,2020-02-19T19:33:57Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-02-19,1.322315,74.02
8,8,2249838395290890,309.98,2020-03-13T12:15:51Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-13,1.385897,429.60
9,9,7638742701884072,35.39,2020-03-08T05:28:01Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-08,1.381052,48.88


In [265]:
df_orders = df_join_q1_fx[['id','customer.id','total_price','total_price_cad','order_date']]
df_orders

,id,customer.id,total_price,total_price_cad,order_date
0,8369263756632563,47178,79.39,109.64,2020-03-07
1,7262170348080494,94720,86.98,120.12,2020-03-01
2,7722055557038194,59933,36.99,51.09,2020-01-01
3,4280852170163518,23955,55.98,76.57,2020-03-10
4,5058732129957127,59933,309.98,428.10,2020-03-15
5,6431399815998774,23955,315.97,436.40,2020-03-12
6,5791957346866373,47178,36.99,53.61,2020-03-24
7,2079640804901496,23955,55.98,74.02,2020-02-19
8,2249838395290890,47178,309.98,429.60,2020-03-13
9,7638742701884072,59933,35.39,48.88,2020-03-08


In [156]:
df_orders.describe()

,id,customer.id,total_price,total_price_cad
count,1.000000e+01,10.000000,10.000000,10.000000
mean,5.203901e+15,52270.500000,112.721000,155.275000
std,2.648302e+15,26256.112944,111.659292,153.962695
min,2.079641e+15,23955.000000,5.990000,7.960000
25%,2.757592e+15,29760.750000,41.737500,58.712500
50%,5.271560e+15,47178.000000,67.685000,95.460000
75%,6.271539e+15,59933.000000,128.965000,169.907500
max,9.786846e+15,94720.000000,315.970000,436.400000


In [157]:
from pandas.io.json import json_normalize

In [158]:
df_order_lines = json_normalize(q1_orders, record_path='line_items',
                               meta = ['id','product_id','product_sku','product_name'],
                               errors='ignore',record_prefix='line_')
df_order_lines.head()


/Users/stephenlang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,line_id,line_product_id,line_product_sku,line_product_name,line_price,id,product_id,product_sku,product_name
0,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40,8369263756632563,NaN,NaN,NaN
1,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,8369263756632563,NaN,NaN,NaN
2,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,7262170348080494,NaN,NaN,NaN
3,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7262170348080494,NaN,NaN,NaN
4,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,7722055557038194,NaN,NaN,NaN


In [159]:
df_order_lines = pd.merge(left=df_order_lines,right= df_join_q1_fx, left_on='id',right_on='id')
df_order_lines['line_price_cad'] = round((df_order_lines['line_price'] * df_order_lines['cad_rates']),2)
df_order_lines_final = df_order_lines[['line_id','id','line_product_id','line_price','line_price_cad']]
df_order_lines_final.head()

,line_id,id,line_product_id,line_price,line_price_cad
0,317808,4280852170163518,477365777,5.99,8.19
1,771854,4280852170163518,694165761,49.99,68.38
2,997208,6431399815998774,278266679,49.99,69.04
3,317808,6431399815998774,477365777,5.99,8.27
4,604665,6431399815998774,510650809,259.99,359.08


In [160]:
df_products = df_order_lines[['line_product_id','line_product_sku','line_product_name']].drop_duplicates()
df_products

,line_product_id,line_product_sku,line_product_name
0,477365777,GOGPR-07,Glueberry OG Pre-Rolls
1,694165761,HYBTINC-03,Hybrid Blend THC Tincture
2,278266679,GPSTASH-01,Grandpa's Stash Dried Flower
4,510650809,PAX-3,Pax 3 Vaporizer
5,573794972,SAGENSOUR-01,Sage N Sour Dried Flower
14,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower


In [166]:
df_customers = df_q1[['customer.id','customer.name','customer.email']].drop_duplicates()
df_customers

,customer.id,customer.name,customer.email
0,47178,William Doe,william.doe@gmail.com
1,94720,Emile Tumson,emile.tumson@gmail.com
2,59933,Ethan Jones,ethan.jones@gmail.com
3,23955,Daniel Smith,daniel.smith@gmail.com


In [162]:
type(df_customers['customer.id'][0])

numpy.int64